In [1]:
import os
os.chdir("../")
%pwd

'd:\\Projects\\Cancer-classfiication-using-MLflow-and-DVC'

In [2]:
# Provide ML credentials
os.environ["MLFLOW_TRACKING_URI"] = ""
os.environ["MLFLOW_TRACKING_USERNAME"] = ""
os.environ["MLFLOW_TRACKING_PASSWORD"] = ""

In [3]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    valid_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from src.ChestCancerClassfication.constants import *
from src.ChestCancerClassfication.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/Data/train",
            valid_data = "artifacts/data_ingestion/Data/valid",
            mlflow_uri = "https://dagshub.com/aryanravula/Cancer-classfiication-using-MLflow-and-DVC.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import json
import mlflow.keras 
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        # No augmentation for validation set
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.valid_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        with open("scores.json", "w") as f:
            json.dump(scores, f)

    def log_into_mlfow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        with mlflow.start_run(nested=True):
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Model registry does not work with the file store
            if tracking_url_type_store != "file":
                #Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name = "InceptionV3")
            else:
                mlflow.keras.log_model(self.model, "model")

    

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlfow()
except Exception as e:
    raise e

[2024-03-16 16:13:17,802: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-16 16:13:17,804: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-16 16:13:17,805: INFO: common: created directory at: artifacts]
Found 72 images belonging to 4 classes.
5/5 [==============================] - 3s 559ms/step - loss: 17.0619 - accuracy: 0.5694


2024/03/16 16:13:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-03-16 16:13:38,955: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 95). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\aryan\AppData\Local\Temp\tmpdksr0ooh\model\data\model\assets
[2024-03-16 16:13:43,462: INFO: builder_impl: Assets written to: C:\Users\aryan\AppData\Local\Temp\tmpdksr0ooh\model\data\model\assets]


Registered model 'InceptionV3' already exists. Creating a new version of this model...
2024/03/16 16:14:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: InceptionV3, version 5
Created version '5' of model 'InceptionV3'.
